# Extração de texto do certificado Udemy

In [ ]:
!pip install pdf2image

In [ ]:
# Biliotecas
import cv2
from glob import glob
import matplotlib.pyplot as plt
import os
import pandas as pd
import re

import pytesseract

import pdf2image
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError




### Funções básicas 

In [ ]:
# Caminho
path = os.getcwd()

# Tesseract
pytesseract.pytesseract.tesseract_cmd = path + r'\Tesseract-OCR\tesseract.exe'
tessdata_dir_config = f'--tessdata-dir "{path}\\Tesseract-OCR\\tessdata"'

# Variaveis
INFO_LIMPO = []
NOME = []
LOGCORROMPIDO = []

# Funcoes Basicas
def verificar_pasta(caminho: str) -> str:
  # Verificar se pasta Repositorio existe

  if os.path.isdir(caminho) == False:
      print(f'A pasta {caminho} não existe. Criando diretório.')
      os.mkdir(caminho)
  else:
      print(f'A pasta {caminho} existe.')


verificar_pasta(path + r'\Repositorio')

pdfs_files = glob(os.path.join(os.getcwd(), path + r'\Repositorio\*.pdf'))

In [ ]:
for fn in range(len(pdfs_files)):
    fn=0

    try:
        # Converter pdf para imagem
        images = convert_from_path(pdfs_files[fn])

    except PDFPageCountError as e:
        print('Nao foi possivel ler o pdf. Corrompido')
        LOGCORROMPIDO.append(pdfs_files[fn])
        pass

    for i in range(len(images)):
        images[i].save('page.png', 'PNG')

    # Leitura da imagem
    img = cv2.imread('page.png', cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    try:

        # Encontrar as palavras na imagem
        boxes = pytesseract.image_to_data(img, lang='por', config=tessdata_dir_config)
        INFO = []  # Armazenar as informacoes obtidas da imagem-pdf

        # Encontrar boxes com as infos
        for a, b in enumerate(boxes.splitlines()):
            print(b)
            if a != 0:
                b = b.split()
                if len(b) == 12:
                    INFO.append(b)
                    #x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                    # cv2.putText(img,b[11],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
                    #cv2.rectangle(img, (x,y), (x+w, y+h), (50, 50, 255), 2)

        # plt.imshow(img)
        # plt.show()

        # Limpando as informacoes dos boxes
        INFO_LIMPO = []
        for i in range(len(INFO)):
            #print(INFO[i][-1])
            INFO_LIMPO.append(INFO[i][-1])

        INFO_LIMPO = ' '.join(INFO_LIMPO)
        print(INFO_LIMPO)

    except :
        print('Nao foi possivel ler o pdf')

    # Nome
    try:
        # Nome do aluno
        NOME.append(re.search(r"certify that (.*[A-Za-z\s]) successfully", INFO_LIMPO).group(1))
    except AttributeError:
        print('Nome nao encotrado')

In [ ]:
df = pd.DataFrame({'Nome': NOME})
df